In [1]:
import pandas as pd
import numpy as np

col_list = ['COMPANY_FKEY','GOING_CONCERN','OPINION_TEXT1','OPINION_TEXT2','OPINION_TEXT3']
bank_data = pd.read_csv('data/Audit Analytics 01.2010.csv', usecols=col_list)
# GOING_CONCERN

/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (178) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [28]:
bank_data.head()

,GOING_CONCERN,COMPANY_FKEY,OPINION_TEXT1,OPINION_TEXT2,OPINION_TEXT3
0,0.0,20.0,Report of Independent Registered Public Accoun...,NaN,NaN
1,0.0,1750.0,Report of Independent Registered Public Accoun...,NaN,NaN
2,0.0,1750.0,Report of Independent Registered Public Accoun...,NaN,NaN
3,0.0,1750.0,Report of Independent Registered Public Accoun...,NaN,NaN
4,0.0,1750.0,Report of Independent Registered Public Accoun...,NaN,NaN


In [33]:
val_counts = bank_data['GOING_CONCERN'].value_counts()
print(val_counts)

op1_nulls = bank_data['OPINION_TEXT1'].isnull().sum() # 166617 items
op2_nulls = bank_data['OPINION_TEXT2'].isnull().sum() # 15 items
op3_nulls = bank_data['OPINION_TEXT3'].isnull().sum() # 3 items
print('Op1: ', op1_nulls)
print('Op2: ', op2_nulls)
print('Op2: ', op3_nulls)


0.0    139699
1.0     26915
Name: GOING_CONCERN, dtype: int64
Op1:  166617
Op2:  166602
Op2:  166614


## Remove all entries that span more than one 'OPINION TEXT'

In [40]:
# get rid of all values with op2 
df_filtered = bank_data[bank_data['OPINION_TEXT2'].isnull()]
df_filtered = df_filtered[['COMPANY_FKEY','GOING_CONCERN','OPINION_TEXT1']]

print(df_filtered)
df_filtered.head()

        COMPANY_FKEY  GOING_CONCERN  \
0               20.0            0.0   
1             1750.0            0.0   
2             1750.0            0.0   
3             1750.0            0.0   
4             1750.0            0.0   
...              ...            ...   
166612     1931055.0            0.0   
166613     1931691.0            1.0   
166614     1933359.0            1.0   
166615     1934945.0            1.0   
166616     1936214.0            0.0   

                                            OPINION_TEXT1  
0       Report of Independent Registered Public Accoun...  
1       Report of Independent Registered Public Accoun...  
2       Report of Independent Registered Public Accoun...  
3       Report of Independent Registered Public Accoun...  
4       Report of Independent Registered Public Accoun...  
...                                                   ...  
166612  Report of Independent Registered Public Accoun...  
166613  REPORT OF INDEPENDENT REGISTERED PUBLIC ACC

,COMPANY_FKEY,GOING_CONCERN,OPINION_TEXT1
0,20.0,0.0,Report of Independent Registered Public Accoun...
1,1750.0,0.0,Report of Independent Registered Public Accoun...
2,1750.0,0.0,Report of Independent Registered Public Accoun...
3,1750.0,0.0,Report of Independent Registered Public Accoun...
4,1750.0,0.0,Report of Independent Registered Public Accoun...


In [37]:
op1_nulls = df_filtered['OPINION_TEXT1'].count() # 166617 items
print(op1_nulls)

166602


In [3]:
# save modified to csv
df_filtered.to_csv('./data/going_concerns.csv')

NameError: name 'df_filtered' is not defined

In [43]:
neg, pos = np.bincount(df_filtered['GOING_CONCERN'])

In [44]:
print('neg: ', neg)
print('pos: ', pos)

neg:  139687
pos:  26915


Remove stopwords and symbols

In [1]:
# import re
# import csv
# import nltk
# from nltk.corpus import stopwords
# nltk.download('stopwords')

# reports = pd.read_csv('./data/going_concerns.csv')

# stop_words = set(stopwords.words('english'))
# pattern = r'[^\w\s]'

# def remove_stopwords(sentence):
#     words = nltk.word_tokenize(sentence)
#     return ' '.join([word for word in words if word.lower() not in stopwords])

# reports []


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/evanaholevas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
import pandas as pd

concern_reports = (pd.read_csv('data/going_concerns.csv'))

In [6]:
# label: GOING_CONCERN (0 or 1)
# split into testing, training, and validation

X = concern_reports.drop(['GOING_CONCERN'], axis=1)
y = concern_reports['GOING_CONCERN']

In [7]:
from sklearn.model_selection import train_test_split

# split data into training (70%), validation (15%), and testing (15%) sets 
X_, X_test, y_, y_test = train_test_split(X, y, train_size=0.8, test_size=0.15, random_state=42, shuffle=True)
X_train, X_validate, y_train, y_validate = train_test_split(X_, y_, train_size=0.82, test_size=0.18, random_state=42, shuffle=True)

print("y_train data distribution:\n", y_train.value_counts())
print("y_validate data distribution:\n", y_validate.value_counts())
print("y_test data distribution:\n", y_test.value_counts())

input_shape = (X.shape[1],)

y_train data distribution:
 0.0    91542
1.0    17748
Name: GOING_CONCERN, dtype: int64
y_validate data distribution:
 0.0    20174
1.0     3817
Name: GOING_CONCERN, dtype: int64
y_test data distribution:
 0.0    21047
1.0     3944
Name: GOING_CONCERN, dtype: int64


## Label Explanation
0=no 1=yes	
Indicates the auditor's opinion contains an explanatory paragraph regarding the going concern assumption.

Auditors include an explanitory paragraph when they concluded there is substantial doubt in a company as a 'going concern.'

In [2]:
import os
import random
import shutil

import pandas as pd

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load data from CSV file
data = pd.read_csv('data/going_concerns.csv')

# Shuffle data
data = data.sample(frac=1, random_state=42).reset_index(drop=True)

# Split data into training, testing, and validation sets
train_ratio = 0.8
test_ratio = 0.1
val_ratio = 0.1

num_samples = data.shape[0]
train_cutoff = int(num_samples * train_ratio)
test_cutoff = int(num_samples * (train_ratio + test_ratio))

train_data = data.iloc[:train_cutoff]
test_data = data.iloc[train_cutoff:test_cutoff]
val_data = data.iloc[test_cutoff:]

# Create output directories if they don't exist
output_dir = './going_concern_reports'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

train_dir = os.path.join(output_dir, 'train')
if not os.path.exists(train_dir):
    os.makedirs(train_dir)

test_dir = os.path.join(output_dir, 'test')
if not os.path.exists(test_dir):
    os.makedirs(test_dir)

val_dir = os.path.join(output_dir, 'val')
if not os.path.exists(val_dir):
    os.makedirs(val_dir)

# Convert data to the required directory structure
def convert_data(data, dirname):
    for label in [0, 1]:
        label_data = data[data['GOING_CONCERN'] == label]
        label_dir = os.path.join(dirname, str(label))
        if not os.path.exists(label_dir):
            os.makedirs(label_dir)

        for idx, row in label_data.iterrows():
            filename = f"{row['COMPANY_FKEY']}.txt"
            filepath = os.path.join(label_dir, filename)

            with open(filepath, 'w', encoding='utf-8') as f:
                f.write(row['OPINION_TEXT1'])

convert_data(train_data, train_dir)
convert_data(test_data, test_dir)
convert_data(val_data, val_dir)


In [ ]:
import pandas as pd
import os

# Split the data into training, testing, and validation sets
train_df, test_df, val_df = np.split(concern_reports.sample(frac=1, random_state=42), [int(.6*len(concern_reports)), int(.8*len(concern_reports))])

# Create folders for each label in the train, test and validation sets
for label in ['pos', 'neg']:
    os.makedirs(f'data/going_concerns/train/{label}', exist_ok=True)
    os.makedirs(f'data/going_concerns/test/{label}', exist_ok=True)
    os.makedirs(f'data/going_concerns/val/{label}', exist_ok=True)

# Save the training data to the appropriate folder
for _, row in train_df.iterrows():
    label = 'pos' if row['GOING_CONCERN'] == 0 else 'neg'
    with open(f'data/going_concerns/train/{label}/{row["COMPANY_FKEY"]}.txt', 'w') as f:
        f.write(row['OPINION_TEXT1'])

# Save the testing data to the appropriate folder
for _, row in test_df.iterrows():
    label = 'pos' if row['GOING_CONCERN'] == 0 else 'neg'
    with open(f'data/going_concerns/test/{label}/{row["COMPANY_FKEY"]}.txt', 'w') as f:
        f.write(row['OPINION_TEXT1'])

# Save the validation data to the appropriate folder
for _, row in val_df.iterrows():
    label = 'pos' if row['GOING_CONCERN'] == 0 else 'neg'
    with open(f'data/going_concerns/val/{label}/{row["COMPANY_FKEY"]}.txt', 'w') as f:
        f.write(row['OPINION_TEXT1'])


In [1]:
import tensorflow as tf

for text_batch, label_batch in raw_train_ds.take(1):
  for i in range(3):
    print("Review", text_batch.numpy()[i])
    print("Label", label_batch.numpy()[i])

ValueError: If `subset` is set, `validation_split` must be set, and inversely.

In [19]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words("english"))

for i in range(len(concern_reports)):
    #tokenize the text into words
    words = nltk.word_tokenize(concern_reports[i][3])
    concern_reports[i][3] = []
    filtered_words = [word for word in words if word.lower() not in stop_words]
    concern_reports[i][3] = ' '.join(filtered_words)


In [23]:
concern_reports

array([[0, 20.0, 0.0,
        'Report Independent Registered Public Accounting Firm Board Directors Shareholders K-Tron International , Inc. Subsidiaries audited accompanying consolidated balance sheets K-Tron International , Inc. ( New Jersey corporation ) Subsidiaries January 2 , 2010 January 3 , 2009 , related consolidated statements income , changes shareholders ’ equity , cash flows three fiscal years ended January 2 , 2010 , January 3 , 2009 , December 29 , 2007 . audits basic financial statements included financial statement schedule listed index appearing page S-1 . financial statements financial statement schedule responsibility Company ’ management . responsibility express opinion financial statements financial statement schedule based audits . conducted audits accordance standards Public Company Accounting Oversight Board ( United States ) . standards require plan perform audit obtain reasonable assurance whether financial statements free material misstatement . audit includ